In [43]:
import boto3
import os
import json

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BedrockEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock

In [44]:
# Load the JSON file
with open("aws_credentials.json", "r", encoding="utf-8-sig") as file:
    data = file.read()  # Read the file content
    credentials = json.loads(data) 

# Extract credentials
aws_access_key_id = credentials["aws_access_key_id"]
aws_secret_access_key = credentials["aws_secret_access_key"]

In [45]:
SERVICE_NAME = 'bedrock-runtime'
REGION_NAME = 'ap-south-1'

import boto3
bedrock = boto3.client(
 service_name=SERVICE_NAME,
 region_name=REGION_NAME,
 aws_access_key_id=aws_access_key_id,
 aws_secret_access_key=aws_secret_access_key
 #endpoint_url=f'https://{SERVICE_NAME}.{REGION_NAME}.amazonaws.com'
)

In [46]:
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=bedrock)

In [47]:
def get_response(llm, faiss_index, question):
    prompt_tempalte = """

    Human: Use the following pieces of context to provide a concise answer to the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}

    Assistant:
    """

    PROMPT = PromptTemplate(template=prompt_tempalte, input_variables=["context", "question"])

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    answer =  qa({"query":question})
    return answer['result']

In [48]:
def get_llm():
    llm = Bedrock(
        model_id="mistral.mistral-7b-instruct-v0:2", 
        client=bedrock
    )
    return llm

In [49]:

    faiss_index = FAISS.load_local(
        folder_path="faiss_index",
        embeddings=bedrock_embeddings,
        allow_dangerous_deserialization=True
    )


In [50]:
question="what is discussed in the meeting"

In [51]:
llm=get_llm()

In [52]:
get_response(llm, faiss_index, question)

'1. Deploying to production and preparing a demo\n    2. Reviewing progress on various aspects of the project, including model fine-tuning, prompt adjustments, and data pipeline testing\n    3. Plans for evaluating and improving model accuracy\n    4. Setting up knowledge bases and preparing for upcoming releases and presentations\n    5. Discussions on enabling approximately 200 Hedge employee accounts for testing\n    6. Announcing the availability of the new system for testing during the AI First session\n    7. Updating prompts and deploying to staging\n    8. Setting up agents for data retrieval without embeddings\n    9. Fine-tuning models, adjusting prompts, and ensuring the correct APIs are called to get the right data.\n    10. Discussions on measuring and improving response accuracy.'